In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/invasivespecies/test/1269.jpg
/kaggle/input/invasivespecies/test/623.jpg
/kaggle/input/invasivespecies/test/764.jpg
/kaggle/input/invasivespecies/train/1269.jpg
/kaggle/input/invasivespecies/train/623.jpg
/kaggle/input/invasivespecies/train/2193.jpg
/kaggle/input/invasive-species-monitoring/train_labels.csv.zip
/kaggle/input/invasive-species-monitoring/sample_submission.csv.zip
/kaggle/input/invasive-species-monitoring/test.7z


## Data processing

In [3]:
import glob
from tensorflow.keras.preprocessing.image import *
!unzip -q -o /kaggle/input/invasive-species-monitoring/train_labels.csv.zip


In [7]:
train=pd.read_csv("train_labels.csv")
train

,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1
...,...,...
2290,2291,1
2291,2292,1
2292,2293,1
2293,2294,1


In [11]:
train['path']="/kaggle/input/invasivespecies/train/"+ train['name'].astype(str)+".jpg"   

In [12]:
train #정답 column 이랑 path column 무조건있어야한다 deep learning 에서는 

,name,invasive,path
0,1,0,/kaggle/input/invasivespecies/train/1.jpg
1,2,0,/kaggle/input/invasivespecies/train/2.jpg
2,3,1,/kaggle/input/invasivespecies/train/3.jpg
3,4,0,/kaggle/input/invasivespecies/train/4.jpg
4,5,1,/kaggle/input/invasivespecies/train/5.jpg
...,...,...,...
2290,2291,1,/kaggle/input/invasivespecies/train/2291.jpg
2291,2292,1,/kaggle/input/invasivespecies/train/2292.jpg
2292,2293,1,/kaggle/input/invasivespecies/train/2293.jpg
2293,2294,1,/kaggle/input/invasivespecies/train/2294.jpg


In [14]:
idg = ImageDataGenerator()

In [18]:
train['invasive']=train['invasive'].astype(str)

In [20]:
train_generator=idg.flow_from_dataframe(train, x_col='path', y_col='invasive')


Found 2295 validated image filenames belonging to 2 classes.


In [29]:
from tensorflow.keras import *
from tensorflow.keras.layers import * 

In [31]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0

In [33]:
model= Sequential()
model.add(EfficientNetB0(include_top=False, pooling = 'avg'))
model.add(Dense(2, activation='softmax'))
model.compile(metrics='acc',optimizer='adam',loss='categorical_crossentropy')

In [34]:
model.fit(train_generator)


2024-02-25 11:38:37.895946: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1708861122.895382    1436 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


72/72 [==============================] - 107s 953ms/step - loss: 0.1851 - acc: 0.9355


In [35]:
!unzip -q -o /kaggle/input/invasive-species-monitoring/sample_submission.csv.zip

sub=pd.read_csv("sample_submission.csv")
sub



,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5
...,...,...
1526,1527,0.5
1527,1528,0.5
1528,1529,0.5
1529,1530,0.5


In [36]:
sub['path']="/kaggle/input/invasivespecies/test/"+ sub['name'].astype(str)+".jpg"  
sub

,name,invasive,path
0,1,0.5,/kaggle/input/invasivespecies/test/1.jpg
1,2,0.5,/kaggle/input/invasivespecies/test/2.jpg
2,3,0.5,/kaggle/input/invasivespecies/test/3.jpg
3,4,0.5,/kaggle/input/invasivespecies/test/4.jpg
4,5,0.5,/kaggle/input/invasivespecies/test/5.jpg
...,...,...,...
1526,1527,0.5,/kaggle/input/invasivespecies/test/1527.jpg
1527,1528,0.5,/kaggle/input/invasivespecies/test/1528.jpg
1528,1529,0.5,/kaggle/input/invasivespecies/test/1529.jpg
1529,1530,0.5,/kaggle/input/invasivespecies/test/1530.jpg


In [37]:
test_generator=idg.flow_from_dataframe(sub, x_col='path', y_col=None, shuffle=False, class_mode=None )



Found 1531 validated image filenames.


In [38]:
result=model.predict(test_generator)

48/48 [==============================] - 43s 852ms/step


In [40]:
sub=pd.read_csv("sample_submission.csv")
sub

,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5
...,...,...
1526,1527,0.5
1527,1528,0.5
1528,1529,0.5
1529,1530,0.5


In [42]:
result #항상오른쪽에 있는 column 에 있는 값을 제출한다
sub['invasive']=result[:,1]

In [43]:
sub.to_csv('sub123.csv',index=0)